<a href="https://colab.research.google.com/github/Sheikthami/VEngage_Assessment/blob/main/VEngage_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assessment for VEngage**


# **Importing Libraries**

In [11]:
pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.6 MB/s eta 0:00:00


In [12]:
import pandas as pd
from faker import Faker
from prettytable import PrettyTable
import csv

In [13]:
fake = Faker()

# **Conversion of data into DataFrame**

In [14]:
data = {
    'Name': [fake.name() for _ in range(100)],
    'email': [fake.email() for _ in range(100)],
    'Phone 1': [fake.phone_number() for _ in range(100)],
    'Phone 2': [fake.phone_number() for _ in range(100)],
}

In [15]:
df = pd.DataFrame(data)
df.to_csv('/phone_book.csv', index=False)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     100 non-null    object
 1   email    100 non-null    object
 2   Phone 1  100 non-null    object
 3   Phone 2  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [17]:
class PhoneBook:
    def __init__(self, records):
        self.records = records

    def select_all(self, limit=10):
        return self.records[:limit]

    def add_record(self, name, email, phone1, phone2):
        new_record = {
            'Name': name,
            'email': email,
            'Phone 1': phone1,
            'Phone 2': phone2
        }

        self.records.append(new_record)
        return new_record

    def remove_records(self, column_name, value):
        deleted_records = [record for record in self.records if record.get(column_name) == value]

        self.records = [record for record in self.records if record.get(column_name) != value]

        return deleted_records

# **Function for reading records from csv file**

In [18]:
def read_records_from_csv(file_name):
    file_path = f'./{file_name}'
    records = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            records.append(row)
    return records

# **2.1 Parser for SELECT ALL statement**

In [19]:
def parse_sql(statement):
    tokens = statement.split()
    if len(tokens) >= 4 and tokens[0].upper() == 'SELECT' and tokens[1] == '*' and tokens[2].upper() == 'FROM':
        table_name = tokens[3]
        if table_name == 'phone_book':
            limit = int(tokens[4]) if len(tokens) > 4 else 10
            result = phone_book.select_all(limit)

            # Create a PrettyTable instance
            table = PrettyTable()

            # Add columns based on the keys of the first record
            if result:
                table.field_names = result[0].keys()

                # Add rows to the table
                for record in result:
                    table.add_row(record.values())

                # Print the table
                print(table)
            else:
                print("No records found.")
        else:
            print(f"Unknown table: {table_name}")
    else:
        print("Invalid SELECT statement.")

# **2.2 Parser for SELECT WHERE statement**

In [20]:
def parse_sql_2(statement):
    tokens = statement.split()
    if len(tokens) >= 4 and tokens[0].upper() == 'SELECT' and tokens[1] == '*' and tokens[2].upper() == 'FROM':
        table_name = tokens[3]
        if table_name == 'phone_book':
            if 'WHERE' in tokens and len(tokens) >= 6:
                parse_where_clause(tokens)
            else:
                limit = int(tokens[-1]) if len(tokens) > 4 else 10
                result = phone_book.select_all(limit)

                # Create a PrettyTable instance
                table = PrettyTable()

                # Add columns based on the keys of the first record
                if result:
                    table.field_names = result[0].keys()

                    # Add rows to the table
                    for record in result:
                        table.add_row(record.values())

                    # Print the table
                    print(table)
                else:
                    print("No records found.")
        else:
            print(f"Unknown table: {table_name}")
    else:
        print("Invalid SELECT statement.")

def parse_where_clause(tokens):
    where_index = tokens.index('WHERE')
    condition = ' '.join(tokens[where_index + 1:])


    condition_parts = condition.split('=')
    if len(condition_parts) == 2:
        column_name = condition_parts[0].strip()
        value = condition_parts[1].strip().strip("';")

        result = phone_book.select_all()

        # Filter records based on the condition
        filtered_records = [record for record in result if record.get(column_name) == value]

        # Create a PrettyTable instance
        table = PrettyTable()

        # Add columns based on the keys of the first record
        if filtered_records:
            table.field_names = filtered_records[0].keys()

            # Add rows to the table
            for record in filtered_records:
                table.add_row(record.values())

            # Print the table
            print(table)
        else:
            print(f"No records found for {column_name}='{value}'.")
    else:
        print("Invalid WHERE condition.")


# **2.3 Parser for INSERT INTO statement**

In [21]:
def parse_sql_3(statement, phone_book):
    tokens = statement.split()
    if len(tokens) >= 6 and tokens[0].upper() == 'INSERT' and tokens[1].upper() == 'INTO':
        table_name = tokens[2]
        if table_name.lower() == 'phone_book' and tokens[3].upper() == 'VALUES':
            # Extract values from the statement
            values_str = ' '.join(tokens[4:])

            # Remove leading '(' and trailing ');'
            values_str = values_str.lstrip("('").rstrip("');")

            # Split values and remove single quotes
            values = [value.strip("'") for value in values_str.split(',')]

            # Assuming your PhoneBook class has a method add_record
            added_record = phone_book.add_record(*values)

            if added_record:
                print(f"Record added successfully: {added_record}")
            else:
                print("Failed to add record. Check your values.")
        else:
            print(f"Unknown table or invalid syntax: {statement}")
    else:
        print("Invalid INSERT INTO statement.")

# 2.4 Parser for DELETE **statement**

In [22]:
def parse_sql_4(statement):
    tokens = statement.split()
    if len(tokens) >= 6 and tokens[0].upper() == 'DELETE' and tokens[1].upper() == 'FROM':
        table_name = tokens[2]
        if table_name == 'phone_book' and tokens[3].upper() == 'WHERE':
            # Extract conditions after WHERE
            conditions = ' '.join(tokens[4:])

            # Assuming the condition is in the form "column='value';"
            if conditions.endswith(';'):
                conditions = conditions[:-1]  # Remove the trailing semicolon

            condition_parts = conditions.split('=')
            if len(condition_parts) == 2:
                column_name = condition_parts[0].strip()
                value = condition_parts[1].strip()

                # Check if the value is enclosed in single quotes and remove them
                if value.startswith("'") and value.endswith("'"):
                    value = value[1:-1]

                # Perform the delete operation
                deleted_records = phone_book.remove_records(column_name, value)

                if deleted_records:
                    print(f"{len(deleted_records)} record(s) deleted successfully.")
                else:
                    print(f"No records found for {column_name}='{value}'.")
            else:
                print("Invalid WHERE condition.")
        else:
            print(f"Unknown table or invalid syntax: {statement}")
    else:
        print("Invalid DELETE statement.")